In [ ]:
%load_ext autoreload
%autoreload 2

import sys; print('Python %s on %s' % (sys.version, sys.platform))

home_or_office = "home_desktop"

import sys
if home_or_office =="home_desktop":
    sys.path.extend(['C:\\Code\\Libraries', 'C:/Code/Libraries'])
elif home_or_office == "home_laptop":
    sys.path.extend(['C:\\Users\\HKGL003\\Documents\\GitHub\\Libraries'])
else:
    sys.path.extend(['C:\\Users\\troy\\Documents\\Code\\Libraries', 'C:\\Users\\troy\\Documents\\Excel Sheets', 
                     'C:/Users/troy/Documents/Code/Libraries'])


from datetime import date, datetime, timedelta
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt

import QuantLib as ql
import dash_bootstrap_components as dbc
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px

from tia.bbg import LocalTerminal

from FinAPI.BondFut.BondFuture_quantlib import USBondFuturesSingleCTDPricerQL, USDRollingBondFuturesQL

Python 3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)] on win32


### Snippets

In [5]:
# Common code snippets

# -------------------------------------------------------------------------------------------------
# profiling a function
# -------------------------------------------------------------------------------------------------

import cProfile, pstats

def main():
    # put your code here
    return objFut.skip_adding_live_value_to_history().fwd_yield_history

profiler = cProfile.Profile()
profiler.enable()
main()
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

# -------------------------------------------------------------------------
# Supress exceptions silently
# -------------------------------------------------------------------------

from contextlib import suppress

def func_which_can_raise_exception(x):
    if x is not None:
        return 1
    else:
        raise Exception("x is None")

def f(x):
    with suppress(Exception): # can be more specfic with the exception type
        func_which_can_raise_exception(x)
v = f(None)  # due to exception, the return value will be None
v is None

         1244 function calls (1223 primitive calls) in 0.014 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.014    0.007 c:\ProgramData\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3424(run_code)
        2    0.000    0.000    0.014    0.007 {built-in method builtins.exec}
        1    0.000    0.000    0.014    0.014 C:\Users\tamaghna\AppData\Local\Temp\ipykernel_13424\4108189913.py:1(<module>)
        1    0.000    0.000    0.014    0.014 C:\Users\tamaghna\AppData\Local\Temp\ipykernel_13424\4108189913.py:6(main)
        1    0.001    0.001    0.014    0.014 C:\Code\Libraries\FinAPI\BondFut\BondFuture_quantlib.py:631(fwd_yield_history)
      4/3    0.000    0.000    0.005    0.002 c:\ProgramData\anaconda3\lib\site-packages\pandas\core\series.py:342(__init__)
        1    0.000    0.000    0.004    0.004 c:\ProgramData\anaconda3\lib\site-packages\pandas\core\series.py:486(

True

In [11]:
# Convert between CuSIP and ISIN 
import string 

def cusip_checksum(cusip_8char):
        """
        Determines the last digit of the 9 digit Cusip based on the 1st 8 characters. A full Cusip code 
        consists of 9 digits. 
            - The first six characters are known as the base (or CUSIP-6), and identify the issuer (company)
            - The seventh and eighth digit identify the exact issue or instrument.
            - The ninth digit is an automatically generated checksum based on the first eight digits.
        
        The checksum is generated using Luhn algorithm
        """
        cusipchars = string.digits + string.ascii_uppercase + "*@#"
        cusip2value = dict((ch, n) for n,ch in enumerate(cusipchars))
        cusipweights = [1,2,1,2,1,2,1,2]
        digits = [(w*cusip2value[ch]) for w,ch in zip(cusipweights, cusip_8char)]
        cs = sum([(x%10+x//10) for x in digits]) % 10
        return str((10-cs)%10)

def isin_checksum(code):
    get_numerical_code = lambda c: str(ord(c) - 55)
    encode_letters = lambda c: c if c.isdigit() else get_numerical_code(c)
    to_digest = ''.join(map(encode_letters, code))

    ints = [int(s) for s in to_digest[::-1]]
    every_second_doubled = [x * 2 for x in ints[::2]] + ints[1::2]

    sum_digits = lambda i: sum(divmod(i,10))
    digit_sum = sum([sum_digits(i) for i in every_second_doubled])

    check_digit = (10 - digit_sum % 10) % 10
    return check_digit

def get_isin_from_cusip(cusip_str, country_code):
    """
    >>> get_isin_from_cusip('037833100', 'US')
    'US0378331005'
    """
    isin_to_digest = country_code + cusip_str.upper()
    check_digit = isin_checksum(isin_to_digest)
    return isin_to_digest + str(check_digit)

cusip_checksum("912828ZQ"), get_isin_from_cusip("912828ZQ6", "US"), 'US912828ZQ64'[2:-1], objFut.curr_ctd_obj.get_isin_from_cusip("912828ZQ", "US")

('6', 'US912828ZQ64', '912828ZQ6', 'US912828ZQ64')

### Guide to features in Version 1

1. Ability to Compute yield, dv01
2. Historical CTDs and historical yields
3. Works for both historical Futures and Current Futures
4. Able to persist necessary data


In [1]:
# Initailize Object

objFut = USBondFuturesSingleCTDPricerQL("TYZ23", True)

NameError: name 'USBondFuturesSingleCTDPricerQL' is not defined

##### Pricing Snapshot

In [8]:
# Test for prcing snapshot - LIVE
objFut.pricing_snapshot()

{'PX_OPEN': 108.34375,
 'PX_HIGH': 108.828125,
 'PX_LOW': 108.28125,
 'PX_LAST': 108.6875,
 'PX_VOLUME': 1313686.0,
 'OPEN_INT': 4753843,
 'FUT_CTD_CUSIP': '91282CHW4',
 'FUT_CNVS_FACTOR': 0.9003,
 'FUT_IMPLIED_REPO_RT': 4.920413177903938,
 'FUT_ACTUAL_REPO_RT': 5.432,
 'CONVENTIONAL_CTD_FORWARD_FRSK': 6.260980698703337,
 'VALUE_DT': Timestamp('2023-09-23 00:00:00'),
 'PRICE': 108.71875}

In [9]:
# Test for prcing snapshot - DATED
data_dt = datetime(2023, 8, 11)
objFut.pricing_snapshot(data_dt)


{'PX_OPEN': 111.265625,
 'PX_HIGH': 111.421875,
 'PX_LOW': 110.65625,
 'PX_LAST': 110.703125,
 'PX_VOLUME': 18744.0,
 'OPEN_INT': 50131.0,
 'FUT_CTD_CUSIP': '91282CHR',
 'FUT_CNVS_FACTOR': 0.8937,
 'FUT_IMPLIED_REPO_RT': 5.21104,
 'FUT_ACTUAL_REPO_RT': 5.423,
 'CONVENTIONAL_CTD_FORWARD_FRSK': 6.3474,
 'VALUE_DT': datetime.datetime(2023, 8, 11, 0, 0),
 'PRICE': 110.703125}

##### Calcualting Yield, fwd dv01 and spot dv01

In [11]:
# Yield Pricing

yld1 = objFut.yld(109+28.5/32) * 100
yld2 = objFut.yld(110) * 100
yld3 = objFut.yld(111) * 100
yld4 = objFut.yld(109) * 100

yld1, yld2, yld3, yld4

(4.309632729147564, 4.292396047939037, 4.135723045555643, 4.450736766780028)

In [55]:
# Clean Price 

print(objFut.cleanPrice(yld1/100) , objFut.cleanPrice(yld2/100), objFut.cleanPrice(yld3/100), objFut.cleanPrice(yld4/100))

# Dirty Price

print(objFut.dirtyPrice(yld1/100) , objFut.dirtyPrice(yld2/100), objFut.dirtyPrice(yld3/100), objFut.dirtyPrice(yld4/100))

109.89062500032448 110.00000000012824 111.00000000014582 108.99999999978208
111.40111051614154 111.5104855159453 112.51048551596288 110.51048551559914


In [45]:
# Test DV01 
#  TYU3
#   109-28+ = [6.4, 6.492]
objFut.fwd_dv01(109+28.5/32), objFut.spot_dv01(109+28.5/32, val_dt = objFut.fut_last_trade_dt)

(6.401719220723056, 6.423877057670288)

##### Simple HIstory - Future Fields (Candle, OI, Volume, CUSIPS, CNVS Fact, Imp Repo, Actual Repo), Curr CTD Fields (Price, Yield)

In [56]:
# Get the history of any field for the Future
field = "PX_LAST" # can be anything that has a history, check in FLDS screen in Bloomberg with the history available flter
objFut.get_hist_values(objFut.futures_ticker, field)

date
2023-03-22    116.781250
2023-03-23    117.671875
2023-03-24    117.718750
2023-03-27    116.593750
2023-03-28    116.265625
                 ...    
2023-09-18    109.515625
2023-09-19    109.125000
2023-09-20    109.218750
2023-09-21    108.421875
2023-09-22    108.687500
Name: TYZ23, Length: 129, dtype: float64

In [57]:
# Direct access properties 

price_hist = objFut.price_history

# for testing, comparing with the manual direct pocess from the previous cell
ts = objFut.get_hist_values(objFut.futures_ticker, "PX_LAST")
if ts is not None: 
    if objFut.add_live_value_to_history:
        snapshot = objFut.pricing_snapshot()
        ts.loc[objFut.today] = snapshot["PX_LAST"]

objFut.add_live_value_to_history, pd.DataFrame({"manual": ts, "object_method" : price_hist})

(True,
                 manual  object_method
 date                                 
 2023-03-22  116.781250     116.781250
 2023-03-23  117.671875     117.671875
 2023-03-24  117.718750     117.718750
 2023-03-27  116.593750     116.593750
 2023-03-28  116.265625     116.265625
 ...                ...            ...
 2023-09-19  109.125000     109.125000
 2023-09-20  109.218750     109.218750
 2023-09-21  108.421875     108.421875
 2023-09-22  108.687500     108.687500
 2023-09-23  108.687500     108.687500
 
 [130 rows x 2 columns])

##### Computed History - CTD objects, CTD Yield Stiched, CTD DV01 Stched, Future Yield, Future DV01, Gross Basis, Net Basis, Imp Repo

In [27]:
# Test HIstorical CTD Cusips
fld = "FUT_CTD_CUSIP"
ts = objFut.get_hist_values(objFut.futures_ticker, fld)
if ts is not None: 
    if objFut.add_live_value_to_history:
        snapshot = objFut.pricing_snapshot()
        ts.loc[objFut.today] = snapshot[fld]
ts.align(objFut.price_history, join='right')[0].fillna(method='ffill').fillna(method='bfill')
df = pd.DataFrame({"Price": objFut.price_history, "CTD": objFut.ctd_cusip_history, "CTD_manual": ts})

In [5]:
objFut.fwd_yield_history

False


2023-03-22    0.034460
2023-03-23    0.033269
2023-03-24    0.033207
2023-03-27    0.034711
2023-03-28    0.035153
                ...   
2023-09-19    0.044309
2023-09-20    0.044160
2023-09-21    0.045431
2023-09-22    0.045006
2023-09-23    0.044956
Length: 130, dtype: float64

In [6]:
objFut.fwd_yield_candles

False
False
False
False
False
False


,OPEN,HIGH,LOW,CLOSE,VOLUME,OPEN_INT
2023-03-22,NaN,NaN,NaN,0.034460,NaN,NaN
2023-03-23,NaN,NaN,NaN,0.033269,NaN,NaN
2023-03-24,NaN,NaN,NaN,0.033207,NaN,NaN
2023-03-27,NaN,NaN,NaN,0.034711,NaN,NaN
2023-03-28,NaN,NaN,NaN,0.035153,NaN,NaN
...,...,...,...,...,...,...
2023-09-19,0.043590,0.043516,0.044358,0.044309,1270702.0,4697001.0
2023-09-20,0.044209,0.043664,0.044931,0.044160,1641379.0,4726628.0
2023-09-21,0.044831,0.044831,0.045706,0.045431,2142465.0,4753843.0
2023-09-22,0.045556,0.044781,0.045656,0.045006,1313686.0,NaN


In [9]:
objFut.add_live_value_to_history = False
computed_hist_dict = dict(fwd_yield = objFut.skip_adding_live_value_to_history().fwd_yield_history, 
                          spot_dv01 = objFut.skip_adding_live_value_to_history().spot_dv01_history, 
                          fwd_dv01 = objFut.skip_adding_live_value_to_history().fwd_dv01_history,
                          ctd_spot_price = objFut.skip_adding_live_value_to_history().ctd_spot_price_history, 
                          ctd_spot_yield = objFut.skip_adding_live_value_to_history().ctd_spot_yield_history, 
                          curr_ctd_price = objFut.skip_adding_live_value_to_history().curr_ctd_price_history, 
                          curr_ctd_yield = objFut.skip_adding_live_value_to_history().curr_ctd_yield_history
                          )
df = pd.DataFrame(computed_hist_dict)

[2023-09-23 17:34:30,272] DEBUG [HistoryManager] [HistoryManager.py] [read_ticker_ts][118] [PID:8492 TID:33968] Cannot find the file C:\Code\Libraries\FinAPI\HistoricalData\Data\AdhocHistory\91282CAE_PX_LAST.csv corresponding to the ticker 91282CAE Govt
[2023-09-23 17:34:30,274] ERROR [HistoryMaager] [HistoryMaager.py] [get_history][582] [PID:8492 TID:33968] 'NoeType' object has o attribute 'ame'
'Traceback (most recet call last):\  File "C:\\Code\\Libraries\\FiAPI\\HistoricalData\\HistoryMaager.py", lie 558, i get_history\    ts.ame = ticker.split()[0]\AttributeError: \'NoeType\' object has o attribute \'ame\''
[2023-09-23 17:34:30,277] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][413] [PID:8492 TID:33968] BloombergHistory: Updating History for bbg ticker: 91282CAE Govt
[2023-09-23 17:34:30,279] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][433] [PID:8492 TID:33968] --- using def_start , start and end dates as 1901-01-01 00:00:00, None and 2023-09-

##### Future Utilities

In [10]:
# Future Utilities to generate past and present Future Codes

from FinAPI.BondFut.FutureUtilities import IMMGenerator as gen

# Example usage
start_date = datetime(2023, 8, 25)
sequence_generator = gen(start_date)

print("Positive Indices:")
for i in range(15):
    print(f"sequence_generator[{i}] = '{sequence_generator[i]}'")

print("\nNegative Indices:")
for i in range(-1, -16, -1):
    print(f"sequence_generator[{i}] = '{sequence_generator[i]}'")

Positive Indices:
sequence_generator[0] = 'U23'
sequence_generator[1] = 'Z23'
sequence_generator[2] = 'H24'
sequence_generator[3] = 'M24'
sequence_generator[4] = 'U24'
sequence_generator[5] = 'Z24'
sequence_generator[6] = 'H25'
sequence_generator[7] = 'M25'
sequence_generator[8] = 'U25'
sequence_generator[9] = 'Z25'
sequence_generator[10] = 'H26'
sequence_generator[11] = 'M26'
sequence_generator[12] = 'U26'
sequence_generator[13] = 'Z26'
sequence_generator[14] = 'H27'

Negative Indices:
sequence_generator[-1] = 'M23'
sequence_generator[-2] = 'H23'
sequence_generator[-3] = 'Z22'
sequence_generator[-4] = 'U22'
sequence_generator[-5] = 'M22'
sequence_generator[-6] = 'H22'
sequence_generator[-7] = 'Z21'
sequence_generator[-8] = 'U21'
sequence_generator[-9] = 'M21'
sequence_generator[-10] = 'H21'
sequence_generator[-11] = 'Z20'
sequence_generator[-12] = 'U20'
sequence_generator[-13] = 'M20'
sequence_generator[-14] = 'H20'
sequence_generator[-15] = 'Z19'


In [11]:
# the sequence generation does not work for the date 25-Aug-2023
dt = datetime(2023, 8, 25)
sequence_generator = gen(start_date)
self = sequence_generator
self.move_date_by_quarters(dt, 1)

date = dt
num_quarters = 1    
new_year = date.year + (date.month + 3 * num_quarters - 1) // 12
for m in range(1, 13):
    print(m, (m + 3 * num_quarters - 1) % 12 + 1, (m + 3 * num_quarters - 1) // 12)
min(31, (11 % 12 + 3) % 12)

1 4 0
2 5 0
3 6 0
4 7 0
5 8 0
6 9 0
7 10 0
8 11 0
9 12 0
10 1 1
11 2 1
12 3 1


2

##### Rolling Futures

In [56]:
from FinAPI.BondFut.BondFuture_quantlib import USBondFuturesSingleCTDPricerQL, USDRollingBondFuturesQL

objRollingFutures = USDRollingBondFuturesQL("TY", datetime(2000, 1, 1))

H00


[2023-09-25 16:50:47,758] ERROR [HistoryManager] [HistoryManager.py] [get_info][994] [PID:8492 TID:33968] FieldError: (H00 COMDTY, FUT_DLV_DT_LAST, BAD_FLD, Field not applicable to security),(H00 COMDTY, FUT_DLV_DT_FIRST, BAD_FLD, Field not applicable to security),(H00 COMDTY, FUT_CNVS_FACTOR, BAD_FLD, Field not applicable to security),(H00 COMDTY, FUT_CTD_ISIN, BAD_FLD, Field not applicable to security),(H00 COMDTY, FUT_FIRST_TRADE_DT, BAD_FLD, Field not applicable to security),(H00 COMDTY, LAST_TRADEABLE_DT, BAD_FLD, Field not applicable to security)


KeyError: 'FUT_CNVS_FACTOR'

In [55]:
objRollingFutures.aggregate_fut_map.keys()

dict_keys(['H00', 'M00', 'U00', 'Z00', 'H01', 'M01', 'U01', 'Z01', 'H02', 'M02', 'U02', 'Z02', 'H03', 'M03', 'U03', 'Z03', 'H04', 'M04', 'U04', 'Z04', 'H05', 'M05', 'U05', 'Z05', 'H06', 'M06', 'U06', 'Z06', 'H07', 'M07', 'U07', 'Z07', 'H08', 'M08', 'U08', 'Z08', 'H09', 'M09', 'U09', 'Z09', 'H10', 'M10', 'U10', 'Z10', 'H11', 'M11', 'U11', 'Z11', 'H12', 'M12', 'U12', 'Z12', 'H13', 'M13', 'U13', 'Z13', 'H14', 'M14', 'U14', 'Z14', 'H15', 'M15', 'U15', 'Z15', 'H16', 'M16', 'U16', 'Z16', 'H17', 'M17', 'U17', 'Z17', 'H18', 'M18', 'U18', 'Z18', 'H19', 'M19', 'U19', 'Z19', 'H20', 'M20', 'U20', 'Z20', 'H21', 'M21', 'U21', 'Z21', 'H22', 'M22', 'U22', 'Z22', 'H23', 'M23', 'U23', 'Z23'])

In [168]:
from FinAPI.BondFut.FutureUtilities import IMMGenerator as gen

def generate_weekdays(start_date):
    current_date = start_date
    while True:
        if current_date.weekday() < 5:  # Monday to Friday
            yield current_date
        current_date += timedelta(days=1)
        
def is_within_T_days(date, T):
    target_months = {3, 6, 9, 12}
    
    for month in target_months:
        target_date = datetime(date.year, month, 1)
        delta = target_date - date
        if 0 < delta.days <= T:
            return 1
    return 0

def generate_fut_map(start_date, T):
    fut_map = {}
    for date in generate_weekdays(start_date):
        if date > datetime.today():
            break
        
        if is_within_T_days(date, T):
            n = 1
        else:
            n = 0
        
        try:
            fut_map[date] = gen(date)[n]
        except Exception as e:
            print(e)
            print(date)
    return fut_map

def code_to_date(data_dict):
    code_ranges = defaultdict(lambda: [datetime.date.max, datetime.date.min])

    for dt, code in data_dict.items():
        code_ranges[code][0] = min(code_ranges[code][0], dt)
        code_ranges[code][1] = max(code_ranges[code][1], dt)

    D = {code: [min_dt, max_dt] for code, (min_dt, max_dt) in code_ranges.items()}
    
    return D

data_dict = generate_fut_map(datetime(2000, 1, 1), 7)
roll_ts = pd.Series(data_dict).sort_index()

code_ranges = defaultdict(lambda: [datetime.max, datetime.min])

for dt, code in data_dict.items():
    code_ranges[code][0] = min(code_ranges[code][0], dt)
    code_ranges[code][1] = max(code_ranges[code][1], dt)

D = {code: [min_dt, max_dt] for code, (min_dt, max_dt) in code_ranges.items()}



{'H00': [datetime.datetime(2000, 1, 3, 0, 0),
  datetime.datetime(2000, 2, 22, 0, 0)],
 'M00': [datetime.datetime(2000, 2, 23, 0, 0),
  datetime.datetime(2000, 5, 24, 0, 0)],
 'U00': [datetime.datetime(2000, 5, 25, 0, 0),
  datetime.datetime(2000, 8, 24, 0, 0)],
 'Z00': [datetime.datetime(2000, 8, 25, 0, 0),
  datetime.datetime(2000, 11, 23, 0, 0)],
 'H01': [datetime.datetime(2000, 11, 24, 0, 0),
  datetime.datetime(2001, 2, 21, 0, 0)],
 'M01': [datetime.datetime(2001, 2, 22, 0, 0),
  datetime.datetime(2001, 5, 24, 0, 0)],
 'U01': [datetime.datetime(2001, 5, 25, 0, 0),
  datetime.datetime(2001, 8, 24, 0, 0)],
 'Z01': [datetime.datetime(2001, 8, 27, 0, 0),
  datetime.datetime(2001, 11, 23, 0, 0)],
 'H02': [datetime.datetime(2001, 11, 26, 0, 0),
  datetime.datetime(2002, 2, 21, 0, 0)],
 'M02': [datetime.datetime(2002, 2, 22, 0, 0),
  datetime.datetime(2002, 5, 24, 0, 0)],
 'U02': [datetime.datetime(2002, 5, 27, 0, 0),
  datetime.datetime(2002, 8, 23, 0, 0)],
 'Z02': [datetime.datetime(20

In [114]:
from collections import defaultdict

# Given dictionary of {dt: code}
data_dict = {
    datetime(2023, 8, 15): 'A',
    datetime(2023, 8, 20): 'B',
    datetime(2023, 8, 10): 'A',
    datetime(2023, 8, 25): 'C',
    datetime(2023, 8, 5): 'B',
    datetime(2023, 8, 1): 'C',
}

# Constructing dictionary D = {code: [min_dt, max_dt]}
code_ranges = defaultdict(lambda: [datetime.max, datetime.min])

for dt, code in data_dict.items():
    code_ranges[code][0] = min(code_ranges[code][0], dt)
    code_ranges[code][1] = max(code_ranges[code][1], dt)

D = {code: [min_dt, max_dt] for code, (min_dt, max_dt) in code_ranges.items()}

print(D)


{'A': [datetime.datetime(2023, 8, 10, 0, 0), datetime.datetime(2023, 8, 15, 0, 0)], 'B': [datetime.datetime(2023, 8, 5, 0, 0), datetime.datetime(2023, 8, 20, 0, 0)], 'C': [datetime.datetime(2023, 8, 1, 0, 0), datetime.datetime(2023, 8, 25, 0, 0)]}


### Sratch

In [33]:
from FinAPI.HistoricalData.HistoryManager import BloombergHistory

df = BloombergHistory.get_intradaybar_history(objFut.futures_ticker, 'TRADE', 1,
                                                     def_start=objFut.static_args['FUT_FIRST_TRADE_DT'],
                                                      update_from_bbg=True)
# resp = LocalTerminal.get_intraday_bar(objFut.futures_ticker, event='TRADE', start=objFut.static_args['FUT_FIRST_TRADE_DT'], end=None, 
#                                      interval=1)
# df2 = resp.as_frame()
df.tail()


In [95]:
from tia.bbg import LocalTerminal, BbgDataManager

mgr = BbgDataManager()
basestring = str
mgr.get_historical('TYU23 Comdty', 'PX_LAST', datetime(2023, 5, 1), datetime(2023, 5, 21))
# Test for hist values
# Test for invidual historical properties
# Test for intraday candles


,PX_LAST
date,
2023-05-01,115.281250
2023-05-02,116.359375
2023-05-03,116.734375
2023-05-04,117.406250
2023-05-05,116.546875
2023-05-08,115.984375
2023-05-09,116.015625
2023-05-10,116.750000
2023-05-11,116.906250


In [4]:
import re

def parse_statement(statement):
    parsed_output = []

    # Regular expression patterns
    pattern = r"(\w+)\((\w+)\)\s+([\d.]+)/([\d.]+)/([\d.]+)\s+(\w+)\s+(\w+)\s+(\d+)k\s+@\s+([\d.]+)"

    matches = re.findall(pattern, statement, re.IGNORECASE)
    
    for match in matches:
        undl, struct, strikes, direction, size, price = match
        strikes = [float(s) for s in strikes.split('/')]
        price = float(price)

        parsed_output.append({
            "undl": undl,
            "struct": struct,
            "opt_typ": "put",
            "strike_1": strikes[0],
            "strike_2": strikes[1],
            "strike_3": strikes[2],
            "direction": direction,
            "size": f"{size}k",
            "price": (strikes[0] + strikes[1] + strikes[2]) / 3
        })

    return parsed_output

# Example statements
statements = [
    "SFRM4(JUN) 94.56/43/31p fly with 94.81/68/56 p fly  ppr pays 2.25-2.5 on 5k all day",
    "SFRU3(sep) 94.56/43/31 p fly ppr sells 5k @ 1.75",
    "0QV3(oct) 95.62/12/94.62 p ladder ppr pays 11 on 5k"
]

# Parse and print parsed output for each statement
for idx, statement in enumerate(statements, start=1):
    parsed = parse_statement(statement)
    print(f"Parsed Output {idx}:")
    for p in parsed:
        print(p)

Parsed Output 1:
Parsed Output 2:
Parsed Output 3:
